In [1]:
########################### Libraries ####################################
import astropy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
import scipy
from scipy.signal import argrelextrema
import matplotlib.backends.backend_pdf
from gatspy import datasets, periodic
from scipy.signal import find_peaks
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn.metrics import accuracy_score
import timeit
from astropy.timeseries import LombScargle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss  
from sklearn.metrics import fbeta_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

### df1 - homogenuous set in zg , df2 - homogenuous set in zr, df3 - overall (zg(min)+zr(min))

In [10]:
df1=pd.read_csv('df1_homogen_zg.csv')
df2=pd.read_csv('df1_homogen_zr.csv')
df3=pd.read_csv('df1_homogen_summ.csv')

In [11]:
df1=df1[df1['Nobs']>30]
df1=df1.iloc[:10000]
df1=df1.reset_index()

df2=df2[df2['Nobs']>30]
df2=df2.iloc[:10000]
df2=df2.reset_index()


df3=df3[df3['Nobs']>30]
df3=df3.iloc[:10000]
df3=df3.reset_index()

### Features - features generation, df- homogenuous df with 'Name' columns (names of files in the folder)

### Metrics - function to receive final results

In [14]:
def Features(type1,df):
    start = timeit.default_timer()
    df['period-0']=0.
    df['power-0']=0.
    df['period-0']=0.
    df['mean_magn']=0.
    df['amplitude']=0.
    df['trand_angle']=0.
    df['skew']=0.
    df['std']=0.
    df['target']=0
#     df=df.iloc[:n]
    names=df['Name'].tolist()
    for i in range(df.shape[0]):
        if df['Type'][i]==type1:
            df['target'][i]=1
        else:
            df['target'][i]=0
    # model = periodic.LombScargleAstroML(fit_period=False)    
    # model.optimizer.set(period_range=(1, 70), first_pass_coverage=30)    

    k=0
    for item in names:
        try:
            with open(r"C:\Users\rtut6\Desktop\Backup\res\{}.txt".format(item)) as file:
                res = file.read().replace("'", '"')
                res=json.loads(res)
                obj_mjd=[]
                obj_mag=[]
                obj_magerr=[]

                for obj in res.keys():
                    for ii in res[obj]['lc']:
                        obj_mjd.append(ii['mjd'])
                        obj_mag.append(ii['mag'])
                        obj_magerr.append(ii['magerr'])

                    frequency, power = LombScargle(obj_mjd, obj_mag, obj_magerr).autopower()

                    df['period-0'][k]=1/frequency[np.argmax(power)]
                    df['power-0'][k]=max(power)
                    df['mean_magn'][k] = np.array(obj_mag).mean()
                    df['amplitude'][k] = (np.array(obj_mag).max() - np.array(obj_mag).min())/2
                    df['trand_angle'][k] = np.polyfit(np.array(obj_mjd), np.array(obj_mag), 1)[0] 
                    df['skew'][k] = scipy.stats.skew(np.array(obj_mag))
                    df['std'][k] = np.array(obj_mag).std()
                    k=k+1
    #               indices = find_peaks(power)
    #               model.fit(obj_mjd, obj_mag, obj_magerr)
    #               for i in range(3):
    #                   df['period-'+str(i)][k]=np.round(model.find_best_periods(n_periods=5,return_scores=True)[0][i],3)
    #                   df['power-'+str(i)][k]=power[int(indices[0][i])]

        except:
            print(1)
    return df


def Metrics(df):
    n=sum(df['target'].tolist())
    dff=df[df['target']==0].iloc[:n]
    df=df[df['target']==1]
    r=[df,dff]
    df=pd.concat(r)
    print(df.shape)
    
    
    X = np.array(df[['period-0','power-0','mean_magn','amplitude']])
    Y = np.array(df['target'])
    validation_size = 0.3
    seed = 2
    X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

    model = XGBClassifier(random_state=34, seed=7, colsample_bytree=0.8, subsample=0.6, n_estimators=700)
    model.fit(X_train,Y_train)
    pred = model.predict(X_validation)
    Metrics=pd.DataFrame(columns=['Accuracy','precision','Roc','F1','Fbeta'],index=range(1))
    Metrics['Roc'][0]=roc_auc_score(pred.tolist(), Y_validation.tolist())
    Metrics['Accuracy'][0]=accuracy_score(Y_validation.tolist(), pred.tolist())
    Metrics['precision'][0]=precision_score(Y_validation.tolist(), pred.tolist())
    Metrics['F1'][0]=f1_score(Y_validation.tolist(), pred.tolist())
    Metrics['Fbeta'][0]=fbeta_score(Y_validation.tolist(), pred.tolist(),beta=0.5)
    return Metrics

In [ ]:
df1=Features('DCEP',df1)
df2=Features('DCEP',df2)
df3=Features('DCEP',df3)

C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: Settin

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [8]:
M1=Metrics(df1)
M2=Metrics(df2)
M3=Metrics(df2)

(234, 18)


,Accuracy,precision,Roc,F1,Fbeta
0,0.661972,0.710526,0.658293,0.692308,0.703125
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [ ]:
model = periodic.LombScargleAstroML(fit_period=False)
model.optimizer.set(period_range=(1, 70), first_pass_coverage=90)
model.fit(obj_mjd, obj_mag, obj_magerr)
model.find_best_periods(n_periods=5,return_scores=True)
# model.periodogram(periods=True)
# periods = np.linspace(0.1, 10, 100)

# omega=2*np.pi/periods
# plt.scatter(scipy.signal.lombscargle(obj_mjd, obj_mag, omega), periods)

In [ ]:
from astropy.timeseries import LombScargle
frequency, power = LombScargle(obj_mjd, obj_mag, obj_magerr).autopower()
frequency[np.argmax(power)]

# plt.ylim(-1, 5)
plt.plot(frequency, power)
plt.ylabel('power')
plt.xlabel('frequency cyclic')
ax = plt.gca()
ax.set_ylim(0,1)
plt.title(obj)
# ax.scale('log')
# plt.ylim(-0.2, 0.6)
# plt.title('')
# print(1/frequency[np.argmax(power)])
# print(1/0.78)
# print(1/1.2)
# print(1/0.41)

In [ ]:
from scipy.signal import find_peaks


indices = find_peaks(power, threshold=0.001)

In [ ]:
indices[0]

In [ ]:
from gatspy.periodic import LombScargleFast
model = LombScargleFast().fit(obj_mjd, obj_mag, obj_magerr)
period, power = model.periodogram_auto(nyquist_factor=200)

#                     D1=np.max(power[peak_data[0]])
#                     ind1=power[peak_data[0]].argmax()
#                     power[peak_data[0][ind1]]=0
#                     D2=np.max(power[peak_data[0]])
#                     ind2=power[peak_data[0]].argmax() 
#                     object_return['max_fs'] =0

Fs= 2*np.pi/(period[power.argmax()])
# Fs1= 2*np.pi/(period[peak_data[0][ind2]])

#### Cumulative curves for Nobs


In [ ]:
# names=df1['Type'].value_counts()[:100].index.tolist()
names=['DCEP','CEP','BCEP','UG','DCEPS','CEP(B)','CEPS','RR']
# pdf = matplotlib.backends.backend_pdf.PdfPages("234.pdf")
for item in names:
    
    data1=df1[df1['Type']==item]['Nobs'].tolist()
    data2=df2[df2['Type']==item]['Nobs'].tolist()
    data3=df3[df3['Type']==item]['Nobs'].tolist()
    
    values1, base1 = np.histogram(data1, bins=60)
    values2, base2 = np.histogram(data2, bins=60)
    values3, base3 = np.histogram(data3, bins=60)
    
    cumulative1 = np.cumsum(values1)
    cumulative2 = np.cumsum(values2)
    cumulative3 = np.cumsum(values3)
    
    #visualisation + saving
    
    f=plt.figure(figsize=(8,5))
    plt.gca().set_xlim([0,160])
    ax = plt.gca()
#     ax.set_ylim([0,2500])
    ax.set_facecolor((1.0, 1, 1))
    plt.plot(base1[:-1], len(data1)-cumulative1, c='green')
    plt.plot(base2[:-1], len(data2)-cumulative2, c='red')
    plt.plot(base3[:-1], len(data3)-cumulative3, c='blue',)
    plt.legend(['ZG','ZR','SUMM'])
    plt.title(item)
    plt.grid(True,linestyle="--")

#     pdf.savefig(f) #saving pdf
    plt.show()
    
# pdf.close()
